In [21]:
import os
import pandas as pd
from tqdm import tqdm
import vqa_logger 
import logging
from pathlib import Path
import datetime
from common import DAL
logger = logging.getLogger(__name__)

In [22]:
df_models = DAL.get_models_data_frame()
df_models = df_models[['class_strategy', 'parameter_count', 'loss_function', 'trainable_parameter_count','activation', 'notes',]]
# 'model_location',
df_scores = DAL.get_partial_scores_data_frame()
df_question_categories = DAL.get_question_categories_data_frame()
df_evaluation_types = DAL.get_evaluation_types_data_frame()

In [27]:
# df_models.tail()

In [28]:
# df_scores.tail()

In [29]:
# df_question_categories

In [30]:
# df_evaluation_types

In [31]:
df = df_models.join(df_scores, how='inner')\
    .join(df_question_categories, on='question_category_id')\
    .join(df_evaluation_types, on='evaluation_type')\
    .sort_index(ascending=False)


df=df[['Category', 'name', 'score','class_strategy',  'loss_function','activation', 'parameter_count','trainable_parameter_count', 'notes']]
df.head()


,Category,name,score,class_strategy,loss_function,activation,parameter_count,trainable_parameter_count,notes
202,Abnormality,wbss,0.079517,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS..."
202,Abnormality,bleu,0.032323,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS..."
202,Abnormality,strict_accuracy,0.012579,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS..."
201,Abnormality,wbss,0.064820,answers,categorical_crossentropy,softmax,20107949,83297,"post_concat_dense_units: (8, 7, 6);\nOptimizer..."
201,Abnormality,bleu,0.020804,answers,categorical_crossentropy,softmax,20107949,83297,"post_concat_dense_units: (8, 7, 6);\nOptimizer..."


### Get data from notes:

In [32]:
new_columns_data = \
{'post_concat_dense_units': None,
'Optimizer': None,
# 'loss': None,
'activation': None,
'prediction ': None,
'lstm_units': None,
'batch_size': None,
'epochs': None,
'class weights': False,
'Inputs Attention': False}

In [33]:
def note_to_lines(raw):
    n = raw
    for c in [';']:
        n = n.replace(c, '')
    lines = (l.strip() for l in n.split('\n'))
    lines = [l for l in lines if ': ' in l]
    return lines


raw = df.notes.values[0]
lines = note_to_lines(raw)
# print('\n'.join(lines))

def get_data_from_lines(lines, col_name):
    default_val = new_columns_data[col_name]
    relevant_lines = [l for l in lines if col_name in l]
    assert len(relevant_lines) <=1
    if len(relevant_lines) == 0:
        data = default_val
    else:
        line = relevant_lines[0]
        data_str = line.split()[-1]
        try:
            data = eval(data_str)            
        except:
            data = data_str
        
    return data
# print('-'*10)        
# for col in new_columns_data:  
#     data = get_data_from_lines(lines, col)
#     print(f'{col}: {data}')


In [34]:
def notes_to_data(notes, col_name):
    line =note_to_lines(notes)
    data = get_data_from_lines(lines, col_name)
    return data

for col in new_columns_data:  
    data = notes_to_data(raw, col)
    print(f'{col}: {data}')

post_concat_dense_units: (8,)
Optimizer: RMSprop
activation: softmax
prediction : answers
lstm_units: 128
batch_size: 32
epochs: 3
class weights: False
Inputs Attention: False


In [35]:
for col in new_columns_data:  
    df[col.replace(' ', '_')] = d.notes.apply(lambda notes: notes_to_data(notes, col))    


In [36]:
df.head()    

,Category,name,score,class_strategy,loss_function,activation,parameter_count,trainable_parameter_count,notes,post_concat_dense_units,Optimizer,prediction_,lstm_units,batch_size,epochs,class_weights,Inputs_Attention
202,Abnormality,wbss,0.079517,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS...","(8,)",RMSprop,answers,128,32,3,False,False
202,Abnormality,bleu,0.032323,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS...","(8,)",RMSprop,answers,128,32,3,False,False
202,Abnormality,strict_accuracy,0.012579,answers,categorical_crossentropy,softmax,20111052,86396,"post_concat_dense_units: (8,);\nOptimizer: RMS...","(8,)",RMSprop,answers,128,32,3,False,False
201,Abnormality,wbss,0.064820,answers,categorical_crossentropy,softmax,20107949,83297,"post_concat_dense_units: (8, 7, 6);\nOptimizer...","(8,)",RMSprop,answers,128,32,3,False,False
201,Abnormality,bleu,0.020804,answers,categorical_crossentropy,softmax,20107949,83297,"post_concat_dense_units: (8, 7, 6);\nOptimizer...","(8,)",RMSprop,answers,128,32,3,False,False
